# Topic Modelling: An Empirical Approach to Theme Detection

Now that we have descriptively explored the 20 newsgroups dataset through corpus linguistics techniques, let's try to automatically detect their themes through an unsupervised machine learning model. Given the fact that we already know how the documents in this corpus are labelled (categorised), it is an excellent way to test the validity of the model - and see how powerful it can be. To use a car metaphor, the algorithm is now in the driver's seat, but we have full control over how it drives and where it is going.

**Topic modelling** is a very popular natural language processing (NLP) technique with applications across a vast variety of academic and industrial domains. JSTOR, for example, uses topic models to restrict search results to specific categories; lawyers and journalists use the technique to better understand immense volumes of legal documents and news stories without having to read all of their content.

In this session, we will not only implement the **Latent Dirichlet Allocation (LDA)** topic model, but also interactively visualise and interpret its results. This interactive visualisation will allow us to adjust the parameters of the model until we arrive at the right optimisation for our corpus.

Topic modelling bibliography: https://mimno.infosci.cornell.edu/topics.html
<br> LDA foundational paper (Blei, Ng, & Jordan 2003): http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf
<br> LDA for English majors: http://www.matthewjockers.net/2011/09/29/the-lda-buffet-is-now-open-or-latent-dirichlet-allocation-for-english-majors/

## Download and import packages and modules

In [ ]:
# https://github.com/amueller/mglearn
!pip install mglearn

In [ ]:
import pandas as pd
import numpy as np
import mglearn
import warnings; warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

%matplotlib inline
import matplotlib.pyplot as plt; plt.style.use('seaborn')
import seaborn as sns
sns.set(style='white')
sns.set(style='whitegrid', color_codes=True)

## Load dataset

We'll use six newsgroup categories this time, which represent six topics. They should be quite distinct as they are chosen from different forum groups (no two are subcategories of the same group). Because we are interested in seeing how the algorithm will describe the documents thematically without being supervised, we will have only one corpus containing all posts from all six topics - remember that the algorithm does not know how many categories are represented. This is the inverse of the approach that we took for corpus linguistics with NLTK, in which we compared the vocabularies of different categories (each category constituted its own corpus).

The topic modelling corpus is one long list of strings, each of which represents a document. This is the input that we will feed into our text preprocessor.

20 newsgroups dataset homepage: http://qwone.com/~jason/20Newsgroups/

In [ ]:
subcategories = ['comp.graphics', 'rec.autos', 'sci.space', 'misc.forsale', 'talk.politics.guns', 'soc.religion.christian']
newsgroups = fetch_20newsgroups(categories=subcategories, remove=('headers', 'footers', 'quotes'))
docs_raw = newsgroups.data
print('Number of documents:', len(docs_raw))

In [ ]:
print(docs_raw[1]) # Second document.

## Text preprocessing and vectorization

Text preprocessing, which comprises tokenization and the removal of certain words and characters, is an extremely important stage of topic modelling. In machine learning terms, it represents a form of feature engineering - different features can dramatically change the results of the topic modelling algorithm.

The input for topic modelling algorithms is a matrix of numbers: strings are not accepted. Thus, we must go a step beyond just tokenizing text (as demonstrated by the corpus linguistics session) to *vectorizing* it. To be more precise, the matrix is a **document-term matrix** - rows are documents (in this case forum posts), and columns are terms (the vocabulary of the corpus). Individual cells contain numbers that represent the frequency of words. Topic modelling operates on a **bag of words (BoW)** approach, which means that word order is not important; only the number of times they occur is needed.

There are two types of frequencies: **term frequency (tf)** and **term frequency-inverse document frequency (tf-idf)**. 
- *Term frequency* is document-level - it is a simple count of the number of times each word appears in each document. This is the most straightforward counting technique, but the problem is that longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.
- *Term frequency-inverse document frequency* takes the entire corpus into account. The most frequent words in the corpus have the lowest idf scores; the least frequent words have the highest idf scores. tf-idf scores balance document frequency (which increases the score) against corpus frequency (which decreases the score). They downscale the weights of words that appear in many documents in the corpus, as these are less informative than those that only occur in a smaller portion of the corpus. They thus highlight words that are more interesting. 
     - Formula: `tf-idf(d, t) = tf(t) * idf(d, t)` 

Scikit-learn's `feature_extraction.text` module contains excellent tools for vectorization: `CountVectorizer` for term frequencies and `TfidfVectorizer` for term frequency-inverse document frequencies. These vectorizers allow for a great degree of vocabulary customisation. 
- We will remove accents, strip stopwords, lowercase all text, only consider alphabetical characters (not numbers), and remove tokens of less than three characters. 
- `max_df` specifies the maximum proportion of documents the word can be in; it thereby represents corpus-specific stopwords. We will only consider words that occur in no more than half of all documents.
- On the other hand, the `min_df` argument spcifies the minimum number of documents the word should be in to be included (i.e., the cut-off). We will only consider words that occur in at least ten documents (which by definition excludes hapax legomena).

`CountVectorizer` documentation: http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
<br>`TfidfVectorizer` documentation: http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [ ]:
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                stop_words='english',
                                lowercase=True,
                                token_pattern=r'\b[A-Za-z]{3,}\b',
                                max_df=0.5, 
                                min_df=10)

Call the `fit()` function to tokenise the text and learn a vocabulary from the documents. The vocabulary is represented by a dictionary with types as keys and numbers as values (each type is assigned a unique and fixed integer ID). I.e., the vectoriser builds a dictionary of feature indices.

In [ ]:
tf_vectorizer.fit(docs_raw)
print(type(tf_vectorizer.vocabulary_))
{w: tf_vectorizer.vocabulary_[w] for w in list(tf_vectorizer.vocabulary_)[:10]} # 10 random vocabulary items and their IDs.

Call the `fit_transform()` function on the documents to encode each as a vector after fitting: tokenizing the text and learning a vocabulary. An encoded vector is as long as the entire vocabulary and has an integer count for the number of times each word appeared in the document. Collectively the vectors comprise a document-term matrix, which is a SciPy sparse matrix - it is sparse because it contains a lot of zeros. 

SciPy sparse matrices don't look very pretty. We can transform the vectors into a dense NumPy array via the `toarray()` function: the array is comprised of lists of numbers, each of which represents one document. We can also get a better look at the matrix by converting it into a pandas dataframe, which is equivalent to an extremely powerful Excel spreadsheet (once you familiarise yourself with pandas, you'll never want to use Excel again!). Given the size of our corpus vocabulary, we'll see loads of zeros.

In [ ]:
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print('Number of documents, number of types:', dtm_tf.shape)
print('Number of tokens:', dtm_tf.sum())

In [ ]:
print(type(dtm_tf))
print(dtm_tf)

In [ ]:
print(dtm_tf.toarray())

In [ ]:
dtm_tf_df = pd.DataFrame(dtm_tf.toarray(), columns=tf_vectorizer.get_feature_names())
dtm_tf_df.head()

Now let's set up the tf-idf vectorizer using the same parameter settings as those of the tf vecorizer and call the `fit_transform()` function on the documents to encode each as a vector. The vectors contain inverse document frequency weightings for all items in the vocabulary.

In [ ]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params()) 
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print('Number of documents, number of types:', dtm_tfidf.shape)

In [ ]:
dtm_tfidf_df = pd.DataFrame(dtm_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names())
dtm_tfidf_df.head()

To get a better sense of what these vectors mean, let's take a look at the words with the highest and lowest idf and tf-idf scores in the corpus, using dataframes and barplots. Remember that the most frequent words in the corpus have the lowest idf scores, and the least frequent words have the highest idf scores. Words with the highest tf-idf scores are frequent in specific documents but not across documents.

We can combine the feature names (word types) from the tf-idf vectorizer with their idf scores into a dictionary, which is easily transformed into a pandas dataframe. The index can be changed to a numerical one as the default index is type, the dictionary key. To display the types with the highest idf scores, we can sort the dataframe by the score in descending order; to see the types with the lowest idf scores, we need to sort in ascending order.

In [ ]:
idf_dict = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer.idf_))
idf_df = pd.DataFrame.from_dict(idf_dict, orient='index').reset_index()
idf_df.columns = ('type', 'idf')
idf_df = idf_df.sort_values(by='idf', ascending=False)
idf_df.head(15)

305 words have the highest idf score: 6.763194. These are the least frequent words in the corpus. Let's plot 15 of them, and then the 15 words with the lowest idf scores.

In [ ]:
idf_df[idf_df.idf>6.763194].shape

In [ ]:
sns.set(font_scale=1.5)
sns.barplot(x='type', y='idf', data=idf_df.head(15))           
plt.title('15 Highest IDF Scores')
plt.xlabel('Type')
plt.ylabel('Weight (IDF Score)')
fig = plt.gcf()
fig.set_size_inches(20, 4)
plt.show()

In [ ]:
idf_df = idf_df.sort_values(by='idf', ascending=True)
sns.barplot(x='type', y='idf', data=idf_df.head(15))           
plt.title('15 Lowest IDF Scores')
plt.xlabel('Type')
plt.ylabel('Weight (IDF Score)')
fig = plt.gcf()
fig.set_size_inches(20, 4)
plt.show()

We can also transform the vocabulary (feature names) into a NumPy array and sort it by idf score to produce the same results as above.

In [ ]:
feature_names = np.array(tfidf_vectorizer.get_feature_names()) 
sorted_by_idf = np.argsort(tfidf_vectorizer.idf_) 
print("Features with highest idf: {}".format(feature_names[sorted_by_idf[-10:]]))
print("Features with lowest idf: {}".format(feature_names[sorted_by_idf[:10]]))

In [ ]:
max_scores = dtm_tfidf.max(axis=0).toarray().ravel() # Maximum tf-idf scores for tokens throughout the entire dataset.
sorted_by_tfidf = max_scores.argsort() 
print("Features with lowest tfidf: {}".format(feature_names[sorted_by_tfidf[:10]]))
print("Features with highest tfidf: {}".format(feature_names[sorted_by_tfidf[-10:]]))

In [ ]:
tfidf_dict = dict(zip(feature_names, max_scores))
tfidf_df = pd.DataFrame.from_dict(tfidf_dict, orient='index').reset_index()
tfidf_df.columns = ('type', 'tfidf')
tfidf_df = tfidf_df.sort_values(by='tfidf', ascending=False)
tfidf_df.head(15)

In [ ]:
sns.barplot(x='type', y='tfidf', data=tfidf_df.head(15))           
plt.title('15 Highest TF-IDF Scores')
plt.xlabel('Type')
plt.ylabel('Weight (TF-IDF Score)')
fig = plt.gcf()
fig.set_size_inches(20, 4)
plt.show()

In [ ]:
tfidf_df = tfidf_df.sort_values(by='tfidf', ascending=True)
sns.barplot(x='type', y='tfidf', data=tfidf_df.head(15))           
plt.title('15 Lowest TF-IDF Scores')
plt.xlabel('Type')
plt.ylabel('Weight (TF-IDF Score)')
fig = plt.gcf()
fig.set_size_inches(22, 4)
plt.show()

## Fit and assess LDA models

Now that we've created our document-term matrices using both tf and tf-idf scores, we can fit our topic model. We will be using the popular **Latent Dirichlet Allocation (LDA)** algorithm, which is probabilistic as opposed to deterministic. There are multiple high-quality LDA implementations in Python, of which Radim Řehůřek's [gensim](https://radimrehurek.com/gensim/) and Pedregosa et al.'s [version in scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) are probably the most well-known. There is also Allen Riddell's excellent [lda](https://github.com/lda-project/lda) library, which implements the model using collapsed Gibbs sampling.

In this tutorial we will use the version in scikit-learn. Riddell's implementation is perhaps better quality but it is quite computationally intensive - because it implements collapsed Gibbs sampling, it requires hundreds (or even thousands) of iterations. We do not need the level of sophistication it represents for two reasons: 1) we are using a simple dataset with very distinctive categories, and 2) we don't have enough time to train the models!

Let's pretend we don't know how many topics there are in these six categories. We can start by looking at ten topics, and adjust the number based upon our interpretation of the results. Topic modelling is a highly iterative process - multiple models need to be fitted in order for the optimal parameters to be determined. It is also by definiton subjective: all the model presents us with are words that it thinks have a high probability of belonging to a topic. We have to interpret what these words mean and assign the topic a label; not all topics are meaningful and can be labelled. Moreover, determining the number of topics represented by a given corpus is both subjective and context-specific: the more granularity we desire, the greater the number of topics we need.

We will apply the LDA algorithm to the tf and tf-idf document-term matrices, to compare their results. Note that tf-idf might not necessarily produce better results, given the fact that our preprocessing is quite extensive. Model fitting and transformation will be performed in one step, because transformation is time-intensive.

In [ ]:
lda_tf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tf_topics = lda_tf.fit_transform(dtm_tf)
print('Number of documents, number of topics:', lda_tf_topics.shape)
print('Number of topics, number of types:', lda_tf.components_.shape)

Let's display the top ten word types that are associated with each of our topics. Each row in `components_` repesents a topic; we'll sort the types in descending order so that those that are most associated will come first. This is achieved through `[:, ::-1]`, which inverts the rows.

In [ ]:
sorted_tf_types = np.argsort(lda_tf.components_, axis=1)[:, ::-1]

mglearn.tools.print_topics(topics=range(10), feature_names=feature_names,
                           sorting=sorted_tf_types, topics_per_chunk=5, n_words=10)

Now let's pick a meaningful topic and display the five documents where the topic is most important.

In [ ]:
cars = np.argsort(lda_tf_topics[:, 8])[::-1]
for i in cars[:5]:
    print(docs_raw[i])
    print('************')

In [ ]:
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf_topics = lda_tfidf.fit_transform(dtm_tfidf)
print('Number of documents, number of topics:', lda_tfidf_topics.shape)
print('Number of topics, number of types:', lda_tfidf.components_.shape)

In [ ]:
sorted_tfidf_types = np.argsort(lda_tfidf.components_, axis=1)[:, ::-1]

mglearn.tools.print_topics(topics=range(10), feature_names=feature_names,
                           sorting=sorted_tfidf_types, topics_per_chunk=5, n_words=10)

In [ ]:
christianity = np.argsort(lda_tfidf_topics[:, 4])[::-1]
for i in christianity[:5]:
    print(docs_raw[i])
    print('************')

## Interactive visualisation with pyLDAvis

pyLDAvis is a very powerful library that facilitates the interpretation of topic model results via an interactive web-based visualisation. It is part of Carson Sievert and Kenny Shirley's R package.

pyLDAvis employs a dimension reduction technique (principal component analysis by default), through which the topics and their distances are plotted in a two-dimensional space. This is displayed on the left of the visualisation. Each circle represents a topic; the area of the circle represents the **topic prevalence** in the corpus. Topic indices (numbers) also indicate prevalence: the largest (most popular) topic is 1; the topic with the lowest number is the least popular. The percentage of tokens that constitute the topic is displayed in the title text on the right side of the visualisation.

The distance between two circles in the plot represents **topic similarity** (the extent to which their vocabularies overlap). This is only an approximation, as we're using a 2D plot to represent the spatial distribution of topics. When we have 10 topics, the similarity matrix has 45 pair-pair distances (`(10 x 9)/2`). This means that we need 45 dimensions to preserve all of the distances! Multidimensional scaling (dimension reduction) tries to preserve the original distances between all of the topics to the greatest possible extent.

On the right, the top 30 **most relevant terms for each topic** are displayed; this aligns with the terms we generated above (although the topic labels are not aligned). The red bar represents the estimated term frequency within the selected topic; the blue bar represents the overall term frequency in the corpus. A longer blue bar indicates that the word is not only relevant for the selected topic, but also for other topics.

To interpret the topics more effectively, we can play around with the value of **lambda (λ)** on the top right slider. When we change this value, the list of words changes. Lambda strikes a balance between choosing only jargon and choosing words that are more layman-oriented. If we want to see more jargon, we should move the slider to the 0 end of the spectrum. Notice that the blue bars disappear here, as we are seeing words that are exclusively relevant for the topic. If we increase the lambda value towards 1, we include terms that are also included in other topics (i.e., more readable words), and the blue bars get longer. 

Note that some 'topics' represent writing style as opposed to substance; they might consist of common words that express general sentiment. These are not meaningful themes.

Some topics could represent subcategories. If we are not interested in granularity, we should reduce the number of topics so that they can be merged.

Another nifty feature of pyLDAvis is that when we hover over a term in the bar chart, the plot shows its **conditional topic distribution**: how will other topics use it? By default, the marginal topic distribution is shown. 

pyLDAvis documentation: https://github.com/bmabey/pyLDAvis

In [ ]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

In [ ]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

### Multidimensional scaling

Scikit-learn's default multidimensional scaling function for the plotting of the intertopic distance map is **principle component analysis**. We can test other scaling functions to see if they improve the results. Several good options are MMDS and TSNE.

MMDS represents Jensen-Shannon Divergence & Metric Multidimensional Scaling; TSNE represents dimension reduction via Jensen-Shannon Divergence & t-distributed Stochastic Neighbor Embedding.

In [ ]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

In [ ]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

## Exercises and Questions

1. Play around with the parameters - number of topics, MDS technique, tf vs. tf-idf. As an unsupervised technique, topic modelling requires a lot of work/interpretation. You have to experiment with multiple settings to determine the right number of topics. By doing this you'll also better understand how the settings work.

2. Which topics are most similar? Which are most distinctive? Can you come up with qualitative reasons for why?